In [1]:
from __future__ import print_function,division
import numpy as np
import nltk
import json
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import time
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
from keras.layers import Embedding
import tensorflow as tf
import cPickle as pickle
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
import gensim
from nmt_utils import *
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from gensim.models import KeyedVectors

Using TensorFlow backend.


In [2]:
reviews = pd.read_csv("ConciseReviews.csv", sep=None,)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
# Inspecting some of the reviews
for i in range(5):
    print("Review #",i+1)
    print("Summary: ",reviews.TargetSummary[i],"\n")
    print("Reviews: ",reviews.ConciseMovieReview[i],"\n\n")
    print()

Review # 1
Summary:  a powerful documentarylike examination of the response to an occupying force the battle of algiers has not aged a bit since its release in 1966 

Reviews:  a riveting reconstruction of the struggle for independence in mid-50s algiers, its pseudo-documentary style still feels as headline-fresh as its content. brilliantly directed set-pieces and remarkable thronging crowd scenes make the film a masterpiece; the ominous familiarity of its subject makes it a must-see. gripping, relevant cinemairish film, "bloody sunday," borrowed its entire structure from "algiers". "the battle of algiers" is a one of a kind masterpiece of pure cinema that you will never forget. the film establishes a kinetic documentary effect, making the impact of every shoot-out and explosion a deeply personal experience. this seminal meticulously crafted work about algiers struggle for freedom won the venic film fest top award, and deservedly garnered pontecorvo oscar nominations as best director a

In [4]:
reviews.TargetSummary[182]

'thanks to energetic performances from its young leads gimme the loot captures a slice of city life with warmth and exuberance'

In [5]:
ExtractedReviews=[]
Summaries=[]
ts=[]
for i in range(1000):
    ExtractedReviews.append(reviews.ConciseMovieReview[i])
    ts.append(reviews.TargetSummary[i])
    Summaries.append('SOS '+reviews.TargetSummary[i]+' EOS')

In [6]:
Summaries[:3]

['SOS a powerful documentarylike examination of the response to an occupying force the battle of algiers has not aged a bit since its release in 1966 EOS',
 'SOS poor plot development and slow pacing keep 54 from capturing the energy of it is legendary namesake EOS',
 'SOS while it hews closely to the 1984 original craig brewer infuses his footloose remake with toetapping energy and manages to keep the story fresh for a new generation EOS']

In [7]:
extractedtokenizer=Tokenizer(char_level=False)
extractedtokenizer.fit_on_texts(ExtractedReviews)

In [8]:
extractedWord_counts=extractedtokenizer.word_counts

In [9]:
extractedWord_counts

OrderedDict([('a', 19949),
             ('riveting', 104),
             ('reconstruction', 3),
             ('of', 17086),
             ('the', 29065),
             ('struggle', 44),
             ('for', 3995),
             ('independence', 15),
             ('in', 8099),
             ('mid', 20),
             ('50s', 13),
             ('algiers', 6),
             ('its', 3380),
             ('pseudo', 9),
             ('documentary', 507),
             ('style', 302),
             ('still', 539),
             ('feels', 447),
             ('as', 4980),
             ('headline', 4),
             ('fresh', 136),
             ('content', 41),
             ('brilliantly', 84),
             ('directed', 260),
             ('set', 289),
             ('pieces', 73),
             ('and', 18026),
             ('remarkable', 113),
             ('thronging', 1),
             ('crowd', 86),
             ('scenes', 280),
             ('make', 860),
             ('film', 4756),
             ('master

In [10]:
print('No of unique words in extracted Reviews:->',len(extractedWord_counts))

No of unique words in extracted Reviews:-> 27788


In [11]:
summarytokenizer=Tokenizer(char_level=False)
summarytokenizer.fit_on_texts(ts)

In [12]:
summaryWord_counts=summarytokenizer.word_counts
summaryWord_index=summarytokenizer.word_index

In [13]:
summaryWord_index['the']

2

In [14]:
len(summarytokenizer.word_counts)

5422

In [15]:
modiefiedSummaryWord_index=summarytokenizer.word_index
print('Initially vocab size for modified summary is:->',len(modiefiedSummaryWord_index))
modiefiedSummaryWord_index['PAD']=0
print('After adding PAD to vocab, vocab size for modified summary is:->',len(modiefiedSummaryWord_index))
modiefiedSummaryWord_index['SOS']=len(summarytokenizer.word_counts)+1
print('After adding SOS to vocab, vocab size for modified summary is:->',len(modiefiedSummaryWord_index))
modiefiedSummaryWord_index['EOS']=len(summarytokenizer.word_counts)+2
print('After adding EOS to vocab, vocab size for modified summary is:->',len(modiefiedSummaryWord_index))


Initially vocab size for modified summary is:-> 5422
After adding PAD to vocab, vocab size for modified summary is:-> 5423
After adding SOS to vocab, vocab size for modified summary is:-> 5424
After adding EOS to vocab, vocab size for modified summary is:-> 5425


In [16]:
summaryWord_index['battle'],modiefiedSummaryWord_index['battle']

(441, 441)

In [17]:
embeddings = dict()
filename = 'glove.6B/glove.6B.100d.txt'
f = open(filename)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings))

Loaded 400000 word vectors.


In [18]:
print('No of unique words in summaries:->',len(summarytokenizer.word_counts))

No of unique words in summaries:-> 5422


In [19]:
ReviewsVocabSize=len(extractedtokenizer.word_index)+1
SummaryVocabSize=len(summarytokenizer.word_index)+1#actually it is 11279
ModifiedVocabSize=len(modiefiedSummaryWord_index)+1
print('ReviewsVocabSize:->',ReviewsVocabSize)
print('SummaryVocabSize:->',SummaryVocabSize)
print('ModifiedVocabSize:->',ModifiedVocabSize)

ReviewsVocabSize:-> 27789
SummaryVocabSize:-> 5426
ModifiedVocabSize:-> 5426


In [20]:
summaryWord_index['SOS']

5423

In [21]:
modiefiedSummaryWord_index['SOS']

5423

In [22]:
# Find the number of words that are missing in Glove, and are used more than our threshold.
missing_words = 0
thresholdR = 20
thresholdS = 0
check=0#words that repeats less than 20 times but are in embedding_index 
most_good_repeating_words=0
badWords=0# words which repeats less than 20 times and also not in embedding_index
for word, count in extractedWord_counts.items():
    if count <thresholdR:
        if word in embeddings:
            check+=1
    if count <thresholdR:
        if word not in embeddings:
            badWords+=1
            
    if count >=thresholdR:
        if word in embeddings:
            most_good_repeating_words+=1
    if count > thresholdR:
        if word not in embeddings:
            #print(word)
            missing_words += 1
            
missing_ratio = round(missing_words/len(extractedWord_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))
print('check:->',check)
print('most_good_repeating_words',most_good_repeating_words)
print('Bad words',badWords)

Number of words missing from CN: 16
Percent of words that are missing from vocabulary: 0.06%
check:-> 22337
most_good_repeating_words 3051
Bad words 2381


In [23]:
extractedtokenizer.word_index

{'woods': 5663,
 'hanging': 4234,
 'woody': 711,
 'comically': 11751,
 'scold': 13354,
 'originality': 1735,
 'appropriation': 11153,
 'bringing': 1319,
 'wooded': 17885,
 'liaisons': 20612,
 'grueling': 5971,
 'wooden': 8270,
 'wednesday': 27325,
 'circuitry': 21069,
 'crotch': 12388,
 'stereotypical': 12888,
 'gwenns': 20499,
 'insular': 21245,
 'splendiferous': 18910,
 'precocity': 17886,
 'sustaining': 13344,
 'consenting': 23155,
 'scraped': 18260,
 'inanimate': 18786,
 'errors': 14590,
 'cooking': 15742,
 'videodrome': 14140,
 'succumb': 6939,
 'shocks': 4509,
 'brainwashed': 24640,
 'perfunctorily': 19799,
 'china': 4127,
 'natured': 2287,
 'kids': 249,
 'uplifting': 1329,
 'climbed': 20588,
 'controversy': 4775,
 'natures': 9022,
 'spotty': 8553,
 'golden': 2030,
 'projection': 25819,
 'outraging': 17466,
 'darkos': 22974,
 'fantasia2000': 14291,
 'stern': 12953,
 'ohorten': 8963,
 'dna': 7896,
 'catchy': 6092,
 'insecurity': 8648,
 'cannibal': 12849,
 'prospector': 23826,
 'mu

In [24]:
embeddingReviews = np.zeros((ReviewsVocabSize, 100))
embedding_dim=100
for word, i in extractedtokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embeddingReviews[i] = embedding_vector
    else:
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings[word]=new_embedding
        embeddingReviews[i]=new_embedding       

In [25]:
len(embeddingReviews)

27789

In [26]:
FN1 = 'embeddingReviewsFilewithOtherdata'
FN0='embeddingReviewsFile'

In [27]:
import cPickle as pickle

In [28]:
with open('%s.pkl'%FN0,'wb') as fp:
    pickle.dump((embeddingReviews),fp,-1)

In [29]:
encodedReviews=extractedtokenizer.texts_to_sequences(ExtractedReviews)
encodedSummaries=summarytokenizer.texts_to_sequences(Summaries)
#encodedTargetSummaries=ts.texts_to_sequences

In [30]:
import psutil

In [31]:
psutil.virtual_memory()

svmem(total=8240300032, available=4668821504, percent=43.3, used=3036299264, free=2330320896, active=3435294720, inactive=2152747008, buffers=1472110592, cached=1401569280, shared=244092928)

In [32]:
encoded_modified_summaries=[]
count=0
for line in Summaries:
    wordList=[]
    count+=1
    #print('count:->',count)
    for word in nltk.word_tokenize(line):
        #print(word)
        if modiefiedSummaryWord_index.has_key(word) !=False:
            wordList.append(modiefiedSummaryWord_index[word])
    encoded_modified_summaries.append(wordList)


In [33]:
print(encoded_modified_summaries[0])

[5423, 3, 60, 2123, 271, 4, 2, 2124, 6, 12, 2125, 1246, 2, 441, 4, 2126, 39, 19, 2127, 3, 537, 883, 9, 2128, 10, 2129, 5424]


In [34]:
testEncodedSummaries=encodedSummaries[950:]
testEncoded_Reviews=encodedReviews[950:]
#testEncoded_modiefiedSummaries=encoded_modified_summaries[3600:]

In [35]:
#valEncodedReviews=encodedReviews[2600:3000]
#valEncodedSummaries=encodedSummaries[2600:3000]
#valEncodedModifiedSummaries=encoded_modified_summaries[2600:3000]

In [36]:
encodedReviews=encodedReviews[:950]
encodedSummaries=encodedSummaries[:950]
encoded_modified_summaries=encoded_modified_summaries[:95]

In [38]:
len(testEncoded_Reviews),len(testEncodedSummaries)#,len(testEncoded_modiefiedSummaries)

(200, 200)

In [39]:
print(encodedSummaries[0])

[3, 60, 2123, 271, 4, 2, 2124, 6, 12, 2125, 1246, 2, 441, 4, 2126, 39, 19, 2127, 3, 537, 883, 9, 2128, 10, 2129]


In [40]:
mydata={'encodedReviews':encodedReviews,'encodedSummaries':encodedSummaries,'encoded_modified_summaries':encoded_modified_summaries,'testEncoded_Reviews':testEncoded_Reviews,'testEncodedSummaries':testEncodedSummaries}
        #'valEncodedReviews':valEncodedReviews,'valEncodedSummaries':valEncodedSummaries,'valEncodedModifiedSummaries':valEncodedModifiedSummaries}

In [41]:
with open('%s.pkl'%FN1,'wb') as fp:
    pickle.dump((mydata),fp,-1)

In [42]:
psutil.virtual_memory()

svmem(total=8240300032, available=5524336640, percent=33.0, used=2275164160, free=3708915712, active=2593849344, inactive=1684021248, buffers=486047744, cached=1770172416, shared=169627648)

In [43]:
maxReviewLength=200
maxSummaryLength=40

In [44]:
paddedReviews = pad_sequences(encodedReviews, maxlen=maxReviewLength, padding='post',truncating='post')
paddedSummary = pad_sequences(encodedSummaries, maxlen=maxSummaryLength, padding='post',truncating='post')
paddedModifiedSummary = pad_sequences(encoded_modified_summaries, maxlen=maxSummaryLength, padding='post',truncating='post')

In [45]:
testPaddedReviews = pad_sequences(testEncoded_Reviews, maxlen=maxReviewLength, padding='post',truncating='post')
testPaddedSummary = pad_sequences(testEncodedSummaries, maxlen=maxSummaryLength, padding='post',truncating='post')
#testPaddedModifiedSummary = pad_sequences(testEncoded_modiefiedSummaries, maxlen=maxSummaryLength, padding='post',truncating='post')

In [46]:
#valPaddedReviews = pad_sequences(valEncodedReviews, maxlen=maxReviewLength, padding='post',truncating='post')
#valPaddedSummary = pad_sequences(valEncodedSummaries, maxlen=maxSummaryLength, padding='post',truncating='post')
#valPaddedModifiedSummary = pad_sequences(valEncodedModifiedSummaries, maxlen=maxSummaryLength, padding='post',truncating='post')

In [47]:
myPaddedData={'paddedReviews':paddedReviews,'paddedSummary':paddedSummary,'paddedModifiedSummary':paddedModifiedSummary,'testPaddedReviews':testPaddedReviews,'testPaddedSummary':testPaddedSummary}
              #,'valPaddedReviews':valPaddedReviews,'valPaddedSummary':valPaddedSummary}

In [48]:
with open('%s.pkl'%'myPaddedDataFile','wb') as fp:
    pickle.dump((embeddingReviews,modiefiedSummaryWord_index,myPaddedData),fp,-1)

In [49]:
print(paddedReviews[290])

[ 4836   807    28     1   691     6  4041   911     3  9796   626     9
  1942  2283  9591     6 10107    43    22    74   558     1    12     5
     2  4958     4    82    54    24   126   328     3     8  1107     2
   835    31   161    22   490    10  5179  1290   140     3   136   644
    29  2198     7  1502 14679     7    16   573  1074    12    44    22
   291    17   263   113   111    22   150  5488     9     1   136   644
  4836   807    92     1    91     4     2  5805  4957    21  1377   219
  1806   259     3  1526  2255  4836   807     5     1  2424     4    19
  3297  5204   566  1390    13   338  1273   486  4836   807    40    17
    23    10  8077    10     1 10107     8  2785    10 11721    62    20
     2 19716   328   206     7     2 10082 10107    16     5     2   409
    73   319   689    21  2169 19717 14680    66   580    12     9  5596
   906    56     4     1  2828     2   178    12    13     8   755     6
   558     2    34    26     2   540  5360    17   

In [50]:
print(paddedSummary[0])

[   3   60 2123  271    4    2 2124    6   12 2125 1246    2  441    4 2126
   39   19 2127    3  537  883    9 2128   10 2129    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]


In [51]:
print(paddedModifiedSummary[0])

[5423    3   60 2123  271    4    2 2124    6   12 2125 1246    2  441    4
 2126   39   19 2127    3  537  883    9 2128   10 2129 5424    0    0    0
    0    0    0    0    0    0    0    0    0    0]


In [74]:
dataIX=paddedReviews
dataTY=paddedSummary
dataIY=paddedModifiedSummary
dataIX.shape,dataIY.shape,dataTY.shape

((2200, 200), (2200, 30), (2200, 30))

In [75]:
psutil.virtual_memory()

svmem(total=8240300032, available=4531535872, percent=45.0, used=3123740672, free=3872231424, active=3329327104, inactive=772489216, buffers=247353344, cached=996974592, shared=321671168)

In [76]:
nb_samples=len(dataIX)
nb_samples

2200

In [77]:
ModifiedVocabSize

11234

In [78]:
decoderInputSummary=to_categorical(paddedModifiedSummary,num_classes=ModifiedVocabSize)

MemoryError: 

In [51]:
decoderInputSummary=decoderInputSummary.reshape(nb_samples,-1,ModifiedVocabSize)

In [52]:
decoderInputSummary.shape

(800, 30, 11234)

In [53]:
psutil.virtual_memory()

svmem(total=8240300032, available=3047186432, percent=63.0, used=4659363840, free=2027376640, active=5140738048, inactive=835387392, buffers=405078016, cached=1148481536, shared=282939392)

In [54]:
decoderTargetSummary=to_categorical(paddedSummary,num_classes=ModifiedVocabSize)

In [55]:
decoderTargetSummary.shape

(24000, 11234)

In [56]:
decoderTargetSummary=decoderTargetSummary.reshape(nb_samples,-1,ModifiedVocabSize)

In [57]:
decoderTargetSummary.shape

(800, 30, 11234)

In [58]:
psutil.virtual_memory()

svmem(total=8240300032, available=1153417216, percent=86.0, used=6554861568, free=145842176, active=7043899392, inactive=810209280, buffers=405168128, cached=1134428160, shared=281202688)

In [49]:
with open('%s.pkl'%'embeddingReviewsFile', 'rb') as fp:
    embeddingReviews = pickle.load(fp)

In [50]:
embedding_dim=100

In [51]:
#Encoder
Encoder_embedding_layer = Embedding(ReviewsVocabSize,
                            embedding_dim,
                            weights=[embeddingReviews],
                            input_length=maxReviewLength,
                            trainable=True)
encoder_input=Input(shape=(maxReviewLength,))
print('encoder_input shape is:->',encoder_input.shape)
embedded_Encoder_inputSequence=Encoder_embedding_layer(encoder_input)
encoder_LSTM=LSTM(256,return_state=True)
encoder_output,encoder_h,encoder_c=encoder_LSTM(embedded_Encoder_inputSequence)
encoder_states=[encoder_h,encoder_c]

encoder_input shape is:-> (?, 200)


In [52]:
decoder_input=Input(shape=(None,ModifiedVocabSize))
#embedded_Decoder_inputSequence=Decoder_embedding_layer(decoder_input)
decoder_LSTM=LSTM(256,return_sequences=True, return_state = True,dropout=0.2)
decoder_output,decoder_h,decoder_c=decoder_LSTM(decoder_input,initial_state=encoder_states)
decoder_dense=Dense(ModifiedVocabSize,activation='softmax')
final_decoder_out=decoder_dense(decoder_output)

In [53]:
model=Model(inputs=[encoder_input,decoder_input],output=final_decoder_out)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [54]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999,decay=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
#filepath="summWithoutAttention.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_weights_only=False, mode='auto', period=1)
checkpointer = ModelCheckpoint(filepath='SummarizationWithoutAttentionV3.2Weights.hdf5', verbose=1, save_best_only=False,mode='auto',period=1)

#callbacks_list = [checkpoint]

In [55]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 200, 100)      4862600     input_1[0][0]                    
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, None, 11234)   0                                            
____________________________________________________________________________________________________
lstm_1 (LSTM)                    [(None, 256), (None,  365568      embedding_1[0][0]                
___________________________________________________________________________________________

In [56]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [ ]:
model.fit(x=[dataIX,decoderInputSummary], 
          y=decoderTargetSummary,
          batch_size=30,
          epochs=400,
          validation_split=0.2,callbacks=[checkpointer])

Train on 640 samples, validate on 160 samples
Epoch 1/400


# Inferencing

In [52]:
#Encoder Inference
encoder_model_inf=Model(inputs=encoder_input,outputs=encoder_states)

In [53]:
#Decoder Inference
decoder_state_input_h=Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]
decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input,initial_state=decoder_input_states)
decoder_states=[decoder_h,decoder_c]
decoder_inf_final_out = decoder_dense(decoder_out)
decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_inf_final_out] + decoder_states )

In [54]:
modiefiedSummaryWord_index['SOS']

4867

In [55]:
int_to_vocab_summaries = {}
for word, value in modiefiedSummaryWord_index.items():
    int_to_vocab_summaries[value] = word

In [60]:
def decode_seq(input_seq):
    # Initial states value is coming from the encoder 
    #We get the encoder states into states_val variable
    states_val = encoder_model_inf.predict(input_seq)#return encoder states
    target_seq = np.zeros((1,1,ModifiedVocabSize))
    print('target_seq shape:->',target_seq.shape)
    target_seq[0, 0, modiefiedSummaryWord_index['SOS']] = 1
    print(target_seq.shape)
    #target_seq=embeddingModifiedSummaries[modiefiedSummaryWord_index['SOS']]
    summarized_sent = ''
    stop_condition = False
    i=1
    while not stop_condition:
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        #print(decoder_out)
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_summary_word = int_to_vocab_summaries[max_val_index]
        #print('sampled_summary_word is:->',sampled_summary_word)
        #print()
        summarized_sent += sampled_summary_word+" "
        #print('summarized_sent is:->',summarized_sent)
        #print()
        if ((sampled_summary_word == 'EOS') or (len(summarized_sent) >= maxSummaryLength)) :
            print('terminated')
            stop_condition = True
        
        target_seq = np.zeros((1,1,ModifiedVocabSize))
        target_seq[0, 0, max_val_index]=1
        
        states_val = [decoder_h, decoder_c]
        i=i+1
        
    return summarized_sent

In [83]:
human_summary=[]
for i in range(50):    
    #print('System Generated Summary:',summary)
    temp=[]
    for j in range(len(paddedSummary[i])):
        temp.append(int_to_vocab_summaries[paddedSummary[i][j]])
    human_summary.append(temp)    
humanSummary=" "        
for i in range(50):
    data=dataIX[i].reshape(1,200)
    summary=decode_seq(data)
    print('System Generated Summary:',summary)
    for j in range(len(human_summary[i])):
        humanSummary+=human_summary[i][j]+" "
    print('Human Summary',humanSummary)
    humanSummary=" "
   

target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: a powerful documentarylike of 
Human Summary  a powerful documentarylike examination of the response to an occupying force the battle of algiers has not aged a bit since its release in 1966 PAD PAD PAD PAD PAD 
target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: though and repetitive and his 
Human Summary  poor plot development and slow pacing keep 54 from capturing the energy of it is legendary namesake PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD 
target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: it is nasty closely to the 1984 
Human Summary  while it hews closely to the 1984 original craig brewer infuses his footloose remake with toetapping energy and manages to keep the story fresh for a new generation PAD PAD 
target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: though acted and an searing in 
Huma

terminated
System Generated Summary: the landmark of hook of a huge 
Human Summary  while fast times at ridgemont high features sean penns legendary performance the film endures because it accurately captured the small details of school work and teenage life PAD PAD PAD 
target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: an longoverdue of japanese perros 
Human Summary  this longoverdue return from alejandro jodorowsky finds him just as overflowing with imagination and heart as fans have come to expect PAD PAD PAD PAD PAD PAD PAD PAD PAD 
target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: a few rousing thriller that that 
Human Summary  a few rousing action sequences cannot make up for pathfinders nonexistent plot and silly dialogue PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD 
target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: though it is a pleasures to reflect 
Human Summar

In [84]:
from nltk.translate.bleu_score import sentence_bleu

In [96]:
humanSummary=" "  
scores=[]
for i in range(50):
    testData=testIX[i].reshape(1,200)
    summary=decode_seq(testData)
    print('System Generated Summary:',summary)
    for j in range(len(human_summary[i])):
        humanSummary+=human_summary[i][j]+" "
    print('Human Summary',humanSummary)
    #calculation of bleu score
    score=sentence_bleu(summary,humanSummary,weights=(0.5, 0.5, 0, 0))
    print('BlEU SCORE IS:->',score)
    scores.append(score)      
    humanSummary=" "

total=0
for i in scores:
    total+=i
print('AVERAGE BLEU SCORE:->',total/len(scores))   

target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: a powerful documentarylike of 
Human Summary  a powerful documentarylike examination of the response to an occupying force the battle of algiers has not aged a bit since its release in 1966 PAD PAD PAD PAD PAD 
BlEU SCORE IS:-> 0.330289129538
target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: though and repetitive and his 
Human Summary  poor plot development and slow pacing keep 54 from capturing the energy of it is legendary namesake PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD 
BlEU SCORE IS:-> 0.313112145543
target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: it is nasty closely to the 1984 
Human Summary  while it hews closely to the 1984 original craig brewer infuses his footloose remake with toetapping energy and manages to keep the story fresh for a new generation PAD PAD 
BlEU SCORE IS:-> 0.303239217432
target_seq shape:-> (1,

target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: though in the heartfelt in sessions 
Human Summary  in addition to its breathtaking underwater photography sharkwater has a convincing impassioned argument of how the plight of sharks affects everyone PAD PAD PAD PAD PAD PAD PAD PAD PAD 
BlEU SCORE IS:-> 0.274351630584
target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: the undiscovered is is a anarchic 
Human Summary  better off dead is an anarchic mix of black humor and surreal comedy anchored by john cusacks winsome charming performance PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD 
BlEU SCORE IS:-> 0.281546253186
target_seq shape:-> (1, 1, 4870)
(1, 1, 4870)
terminated
System Generated Summary: the landmark of hook of a huge 
Human Summary  while fast times at ridgemont high features sean penns legendary performance the film endures because it accurately captured the small details of school work and teenage life PAD PAD PAD 
